In [359]:

import pandas as pd
import numpy as np
import re
import xlsxwriter


In [360]:
def clean_data(data):
    data = data.iloc[7:]
    data = data.iloc[:, 1:]
    data = data.reset_index(drop=True)
    data.columns = data.iloc[0]
    data = data.iloc[1:]
    data = data.rename(columns=lambda x: re.sub(r'\W+', '_', x.strip().lower()))
    return data

In [361]:
data = pd.read_excel(r'D:\6.-Universidad Arturo Prat - Python\Evaluación Docente 2022\Autoevaluación\CE_HONORARIOS_1erSemestre.xlsx')

In [362]:
data = clean_data(data)

In [363]:
agrupacion_coberturas = ['proceso','semestre','sede','tipo_carrera','facultad','codigo_carrera','nombre_carrera','rut_completo','nombres','apaterno','amaterno','contrato','estado']
agrupacion_resultados = ['proceso','semestre','sede','tipo_carrera','facultad','codigo_carrera','nombre_carrera','rut_completo','nombres','apaterno','amaterno','contrato']
estados_sinpendiente = ['finalizado','no_iniciada']
estados = ['finalizado','no_iniciada','pendiente']
nuevos_nombres = {'facultad':'unidad_academica','nombres':'nombre_docente','apaterno':'apaterno_docente','amaterno':'amaterno_docente','rut_completo':'rut_docente'}
columnas_respuesta = [col for col in data.columns if col.startswith('respuesta')]
escala_puntuacion = {'Siempre': 3.0, 'Generalmente': 2.0, 'Pocas veces': 1.0, 'Nunca': 0.0, '--': 0.0}
respuestas_mediador = [col for col in data.columns if col.startswith('respuesta_1_')]
respuestas_gestion = [col for col in data.columns if col.startswith('respuesta_2_')]
columnas_excluidas = ['titulo','grado','sexo','edad','jerarquia','tipo_jornada','inicio_aplicacion','fin_aplicacion','fecha_inicio','fecha_fin']
agrupacion_merge = ['proceso','semestre','sede','tipo_carrera','unidad_academica','codigo_carrera','nombre_carrera','rut_docente','nombre_docente','apaterno_docente','amaterno_docente','contrato']

In [364]:
def coberturas_autoevaluacionhonorarios(data):
    try:
        data_coberturas = data.groupby(agrupacion_coberturas).size().reset_index(name='count')
        data_coberturas = data_coberturas.pivot_table(index=agrupacion_resultados, columns='estado', values='count', fill_value=0).reset_index()
        data_coberturas = data_coberturas.rename(columns=lambda x: re.sub(r'\W+', '_', x.strip().lower()))
        data_coberturas = data_coberturas.rename(columns=nuevos_nombres)
        data_coberturas['total'] = data_coberturas[estados].sum(axis=1)
        data_coberturas['finalizado_percent'] = (data_coberturas['finalizado'] / data_coberturas['total']) * 100
    except KeyError:
        data_coberturas = data.groupby(agrupacion_coberturas).size().reset_index(name='count')
        data_coberturas = data_coberturas.pivot_table(index=agrupacion_resultados, columns='estado', values='count', fill_value=0).reset_index()
        data_coberturas = data_coberturas.rename(columns=lambda x: re.sub(r'\W+', '_', x.strip().lower()))
        data_coberturas = data_coberturas.rename(columns=nuevos_nombres)
        data_coberturas['total'] = data_coberturas[estados_sinpendiente].sum(axis=1)
        data_coberturas['finalizado_percent'] = (data_coberturas['finalizado'] / data_coberturas['total']) * 100
    
    return data_coberturas

In [365]:
def resultados_autoevaluacionhonorarios(data):
    
    data_resultados = data[(data['estado']=="Finalizado")]
    data_resultados[columnas_respuesta] = data_resultados[columnas_respuesta].replace(escala_puntuacion)
    data_resultados = data_resultados.drop(columns=columnas_excluidas)
    data_resultados = data_resultados.groupby(agrupacion_resultados).mean().round(2).reset_index()
    data_resultados = data_resultados.rename(columns=nuevos_nombres)
    
    return data_resultados

# Coberturas de Autoevaluación a Honorarios

In [366]:
ce1 = pd.read_excel(r'D:\6.-Universidad Arturo Prat - Python\Evaluación Docente 2022\Autoevaluación\CE_HONORARIOS_1erSemestre.xlsx')
ce2 = pd.read_excel(r'D:\6.-Universidad Arturo Prat - Python\Evaluación Docente 2022\Autoevaluación\CE_Honorarios_2doSemestre.xlsx')
ptu1 = pd.read_excel(r'D:\6.-Universidad Arturo Prat - Python\Evaluación Docente 2022\Autoevaluación\PTU_HONORARIOS_1erSemestre.xlsx')
ptu2 = pd.read_excel(r'D:\6.-Universidad Arturo Prat - Python\Evaluación Docente 2022\Autoevaluación\PTU_HONORARIOS_2doSemestre.xlsx')
tns1 = pd.read_excel(r'D:\6.-Universidad Arturo Prat - Python\Evaluación Docente 2022\Autoevaluación\TNS_Honorarios_1erSemestre.xlsx')
tns2 = pd.read_excel(r'D:\6.-Universidad Arturo Prat - Python\Evaluación Docente 2022\Autoevaluación\TNS_HONORARIOS_2doSemestre.xlsx')

In [367]:
ce1 = clean_data(ce1)
ce2 = clean_data(ce2)
ptu1 = clean_data(ptu1)
ptu2 = clean_data(ptu2)
tns1 = clean_data(tns1)
tns2 = clean_data(tns2)


In [368]:
ce1_coberturas = coberturas_autoevaluacionhonorarios(ce1)
ce2_coberturas = coberturas_autoevaluacionhonorarios(ce2)
ptu1_coberturas = coberturas_autoevaluacionhonorarios(ptu1)
ptu2_coberturas = coberturas_autoevaluacionhonorarios(ptu2)
tns1_coberturas = coberturas_autoevaluacionhonorarios(tns1)
tns2_coberturas = coberturas_autoevaluacionhonorarios(tns2)

In [ ]:
ce1_resultados = resultados_autoevaluacionhonorarios(ce1)
ce2_resultados = resultados_autoevaluacionhonorarios(ce2)
ptu1_resultados = resultados_autoevaluacionhonorarios(ptu1)
ptu2_resultados = resultados_autoevaluacionhonorarios(ptu2)
tns1_resultados = resultados_autoevaluacionhonorarios(tns1)
tns2_resultados = resultados_autoevaluacionhonorarios(tns2)

In [370]:
Coberturas_Honorarios_Autoevaluacion = pd.concat([ce1_coberturas,ce2_coberturas,ptu1_coberturas,ptu2_coberturas,tns1_coberturas,tns2_coberturas])

In [371]:
Resultados_Honorarios_Autoevaluacion = pd.concat([ce1_resultados,ce2_resultados,ptu1_resultados,ptu2_resultados,tns1_resultados,tns2_resultados])

In [372]:
autoevaluacion_honorarios_2022 = Coberturas_Honorarios_Autoevaluacion.merge(
    right = Resultados_Honorarios_Autoevaluacion,
    how = "outer",
    on = agrupacion_merge
)

In [373]:
autoevaluacion_honorarios_2022[columnas_respuesta] = autoevaluacion_honorarios_2022[columnas_respuesta].fillna(0)

In [374]:
autoevaluacion_honorarios_2022['pendiente']= autoevaluacion_honorarios_2022['pendiente'].fillna(0)

In [375]:
autoevaluacion_honorarios_2022['mediador'] = autoevaluacion_honorarios_2022[respuestas_mediador].mean(axis=1).round(2)

In [376]:
autoevaluacion_honorarios_2022['gestion'] = autoevaluacion_honorarios_2022[respuestas_gestion].mean(axis=1).round(2)

In [377]:
writer = pd.ExcelWriter('Resultados/EvaluaciónDocenteAutoevaluacionHonorarios2022.xlsx', engine='xlsxwriter')
autoevaluacion_honorarios_2022.to_excel(writer, sheet_name='AutoevaluacionHonorarios2022', index=True)
writer.save()

C:\Users\anddiazc\AppData\Local\Temp\ipykernel_11748\2179554248.py:3: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
